In [27]:
%pylab inline
import pandas as pd
import os

from sklearn import linear_model
from sklearn.preprocessing import Imputer

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [28]:
path = '../../../../Data/refined_data/puma50.csv'
puma = pd.read_csv(path, low_memory=False, index_col=0, na_values='.')
puma.shape

(2378, 1900)

In [29]:
yPath = '../../../../Data/refined_data/predict.csv'
ys = pd.read_csv(yPath, low_memory=False, index_col=0, na_values='.')[['internetP', 'label']]
ys.shape

(2378, 2)

In [30]:
def imputeDF(df):
    cols = df.columns
    L = df['label']
    
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    puma_imp = pd.DataFrame(imp.fit_transform(df.drop('label',1)))
    
    puma_imp.index = df.index
    puma_imp.columns = cols[:puma_imp.shape[1]]
    puma_imp['label'] = L
    print puma_imp.shape
    return puma_imp

In [31]:
puma_imp = imputeDF(puma)

(2378, 1900)


In [32]:
#Preprocessing

In [33]:
ip = ys['internetP']/100

In [34]:
#Splitting Data:

In [35]:
Xtest, Xtrain, Xval   = (x[1].drop('label',1) for x in puma_imp.groupby('label'))

In [36]:
ytest, ytrain, yval = (x[1]['internetP'] for x in ys.groupby('label'))

In [37]:
datas = {'Xtest':Xtest, 
         'Xtrain':Xtrain, 
         'Xval':Xval,
         'ytest':ytest,
         'ytrain':ytrain,
         'yval':yval}

### Setting Ridge Model

In [42]:
clf = linear_model.Ridge(alpha=0.5)

In [40]:
def evaluate(model, datas=datas ):
    try:
        x = model.fit(datas['Xtrain'], datas['ytrain'])
        print 'train:', x.score(datas['Xtrain'], datas['ytrain'])
        print 'valid:', x.score(datas['Xval'], datas['yval'])
    except Exception, e:
        print str(e)


In [43]:
evaluate(clf)

train: 0.975041053844
valid: 0.801516313188


C:\Users\Alan\Anaconda\lib\site-packages\sklearn\linear_model\ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


### RidgeCV:

In [44]:
clf = linear_model.RidgeCV(alphas=(0.1,1.0,10.0))

In [45]:
evaluate(clf)

train: -76.4286529602
valid: -71.8939566994


### Lasso

In [70]:
clf = linear_model.Lasso(alpha = 0.9)

In [71]:
evaluate(clf)

train: 0.933532818371
valid: 0.90581317518


### Elastic

In [46]:
clf = linear_model.ElasticNet(positive = True)

In [47]:
evaluate(clf)

train: 0.820514444838
valid: 0.809416139573


### ElasticCV

In [48]:
clf = linear_model.ElasticNetCV(positive = True)

In [49]:
evaluate(clf)

train: 0.461994180793
valid: 0.435728501375


In [50]:
#Choosing Lasso:

In [84]:
clf = linear_model.Lasso(alpha = 0.9)

In [85]:
x = clf.fit(datas['Xtrain'], datas['ytrain'])
print 'train:', x.score(datas['Xtrain'], datas['ytrain'])
print 'valid:', x.score(datas['Xval'], datas['yval'])
print 'test:', x.score(datas['Xtest'], datas['ytest'])

train: 0.933532818371
valid: 0.90581317518
test: 0.89587125692
